In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import wavfile
from scipy.fft import fft, fftfreq
from scipy.signal import spectrogram
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import random
import math

In [2]:
def find_min_samples(folder_path):
    files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.wav')]

    min_samples = float('inf')
    for file in files:
        fs, signal_wave = wavfile.read(file)
        min_samples = min(min_samples, len(signal_wave))

    return min_samples

In [3]:
def save_spectrogramPCA_plot(image, filename, factor):
    plt.figure(figsize=(10,10))

    c = plt.pcolormesh(image, cmap = "viridis")
    plt.axis('off')
    
    tmp = c.get_clim()
    c.set_clim(-10, 10)
    
    plt.savefig(filename, bbox_inches='tight', pad_inches=0)
    plt.close()

In [4]:
def concatenate_signals(f_selected, fs, sig_len, folder_path, train_indices, val_indices, test_indices):
    
    train_split, validation_split, test_split = 0.5, 0.25, 0.25
    shuffle_dataset = True
    
    time = np.linspace(0., sig_len/fs, sig_len)
    signal_audio_train = np.empty((f_selected,10)); signal_audio_val = np.empty((f_selected,10)); signal_audio_test = np.empty((f_selected,10))
    files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.wav')]

    for filename in files:
        _, signal = wavfile.read(filename)
        freq, time, mag = spectrogram(signal[:sig_len], fs, nperseg = 10000, nfft = 50000)
        number = int(filename.split('_')[1].split('.')[0])
        if (number in train_indices):
            signal_audio_train = np.concatenate((signal_audio_train, np.log(mag + 1e-6)[:f_selected,:]), axis = 1)
        elif (number in test_indices):
            signal_audio_test = np.concatenate((signal_audio_test, np.log(mag + 1e-6)[:f_selected,:]), axis = 1)
        elif (number in val_indices):
            signal_audio_val = np.concatenate((signal_audio_val, np.log(mag + 1e-6)[:f_selected,:]), axis = 1)
        
    signal_audio_train = signal_audio_train[:,10:];  signal_audio_val = signal_audio_val[:,10:]; signal_audio_test = signal_audio_test[:,10:]
    return signal_audio_train, signal_audio_val, signal_audio_test

In [5]:
def generate_spectograms(folder_path, save_path, data_train, data_val, data_test, factor, train_indices, val_indices, test_indices):
    files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.wav')]
    cnt_train = 0; cnt_test = 0; cnt_val = 0
    for filename in files:
        audio_name = os.path.splitext(os.path.basename(filename))[0]
        number = int(filename.split('_')[1].split('.')[0])
        label = audio_name.split('_')[0]
        if (number in train_indices):
            savename = f"{save_path}/train/{label}/{audio_name}.png"
            save_spectrogramPCA_plot(data_train[:,0+(cnt_train*10):10+(cnt_train*10)], savename, factor)
            cnt_train += 1
        elif (number in val_indices):
            savename = f"{save_path}/val/{label}/{audio_name}.png"
            save_spectrogramPCA_plot(data_val[:,0+(cnt_val*10):10+(cnt_val*10)], savename, factor)
            cnt_val += 1
        elif (number in test_indices):
            savename = f"{save_path}/test/{label}/{audio_name}.png"
            save_spectrogramPCA_plot(data_test[:,0+(cnt_test*10):10+(cnt_test*10)], savename, factor)
            cnt_test += 1

In [6]:
def run_pca(signal_train, signal_val, signal_test, rate, f_selected):
    scaler = StandardScaler()
    scaler.fit(signal_train.T)
    scaled_signal_train = scaler.transform(signal_train.T); scaled_signal_val = scaler.transform(signal_val.T); scaled_signal_test = scaler.transform(signal_test.T)
    if (rate ==1):
        pcaFit_train = scaled_signal_train.T; pcaFit_val = scaled_signal_val.T; pcaFit_test = scaled_signal_test.T
        pca = []
    else:
        pca = PCA(rate)
        pca.fit(scaled_signal_train)
        pcaFit_train = pca.transform(scaled_signal_train).T; pcaFit_val = pca.transform(scaled_signal_val).T; pcaFit_test = pca.transform(scaled_signal_test).T
    return pcaFit_train, pcaFit_val, pcaFit_test, pca

In [7]:
audio_data = "./data/"
minor_folder_audio = os.path.join(audio_data, "Minor")
major_folder_audio = os.path.join(audio_data, "Major")

In [8]:
min_samples_minor = find_min_samples(minor_folder_audio)
min_samples_major = find_min_samples(major_folder_audio)
overall_min_samples = min(min_samples_minor, min_samples_major)

In [9]:
def get_train_val_test_data(folder):

    train_indices, val_indices, test_indices = [], [], []
    
    train_split, validation_split, test_split = 0.5, 0.25, 0.25
    random.seed(17)
    np.random.seed(17)
    shuffle_dataset = True
    
    files = [os.path.join(folder, file) for file in os.listdir(folder) if file.endswith('.wav')]

    dataset_size = len(files)
    indices = list(range(dataset_size))
    class_indices = [list(range(0,dataset_size))]
    
    for indices in class_indices:
        random.shuffle(indices)
        num_samples = len(indices)
        train_size = int(train_split * num_samples)
        val_size = test_size = int(validation_split * num_samples)
        train_indices.extend(indices[:train_size])
        val_indices.extend(indices[train_size:train_size + val_size])
        test_indices.extend(indices[train_size + val_size:train_size + val_size + test_size])
    return train_indices, val_indices, test_indices

In [10]:
def process_data(f_selected_values, explained_variance_values, fs, overall_min_samples, major_folder_audio, minor_folder_audio, factor):

    major_train_indices, major_val_indices, major_test_indices = get_train_val_test_data(major_folder_audio)
    minor_train_indices, minor_val_indices, minor_test_indices = get_train_val_test_data(minor_folder_audio)

    for f_selected in f_selected_values:

        signal_major_train, signal_major_val, signal_major_test = concatenate_signals(f_selected, fs, overall_min_samples, major_folder_audio,  major_train_indices, major_val_indices, major_test_indices)
        signal_minor_train, signal_minor_val, signal_minor_test = concatenate_signals(f_selected, fs, overall_min_samples, minor_folder_audio,  minor_train_indices, minor_val_indices, minor_test_indices)
        
        for explained_variance in explained_variance_values:

            print(f"Processing f_selected: {f_selected}, explained_variance: {explained_variance}")
            pca_fit_major_train, pca_fit_major_val, pca_fit_major_test, pca_major = run_pca(signal_major_train, signal_major_val, signal_major_test, explained_variance, f_selected)
            pca_fit_minor_train, pca_fit_minor_val, pca_fit_minor_test, pca_minor = run_pca(signal_minor_train, signal_minor_val, signal_minor_test, explained_variance, f_selected)
            
            if (explained_variance == 1):
                images_dir = f"./data/audio-images/PCA_f{f_selected}_expVar{explained_variance}_ncompMinor{f_selected}_ncompMajor{f_selected}"
            else:
                images_dir = f"./data/audio-images/PCA_f{f_selected}_expVar{explained_variance}_ncompMinor{pca_minor.n_components_}_ncompMajor{pca_major.n_components_}"

            folder_names = ["train", "test", "val"]
            for folder_name in folder_names:
                train_subfolder = os.path.join(images_dir, folder_name)
                test_subfolder = os.path.join(images_dir, folder_name)
                val_subfolder = os.path.join(images_dir, folder_name)

                for subfolder in [train_subfolder, test_subfolder, val_subfolder]:
                    minor_folder = os.path.join(subfolder, "Minor")
                    major_folder = os.path.join(subfolder, "Major")
                    for folder in [minor_folder, major_folder]:
                        if not os.path.exists(folder):
                            os.makedirs(folder)

            print("Generating spectograms")
            generate_spectograms(major_folder_audio, images_dir, pca_fit_major_train, pca_fit_major_val, pca_fit_major_test,factor, major_train_indices, major_val_indices, major_test_indices)
            generate_spectograms(minor_folder_audio, images_dir, pca_fit_minor_train, pca_fit_minor_val, pca_fit_minor_test, factor, minor_train_indices, minor_val_indices, minor_test_indices)

In [11]:
f_selected_values = [6500]
explained_variance_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
fs = 44100
factor = 2

In [12]:
process_data(f_selected_values, explained_variance_values, fs, overall_min_samples, major_folder_audio, minor_folder_audio, factor)

Processing f_selected: 6500, explained_variance: 1
Generating spectograms
